<a href="https://colab.research.google.com/github/ravidu-hevaganinge/AI_Scribe/blob/main/wav_2vec_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries and prepare co-lab envirement


this section of code should be run at very of beginning!!


In [ ]:
!pip install torch torchaudio fairseq #Install Required Libraries
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load pre-trained model(wav2vec2) and processor
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are

# used for only one m4a file


used for one m4a file


In [ ]:
#load audio file
audio_path = "/content/drive/MyDrive/UMD BIOE CAPSTONE PROJECT/sample/Raw data/SI(ACH000).m4a"
waveform, sample_rate = torchaudio.load(audio_path) # Load audio data and sample rate from Raw data/SI(ACH000) file
# waveform represents the audio data that has been loaded from the audio file

In [ ]:
# check the sample rate
if sample_rate != processor.feature_extractor.sampling_rate:
    resampler = torchaudio.transforms.Resample(sample_rate, processor.feature_extractor.sampling_rate)
    waveform = resampler(waveform)
# this part used for checking whether the sample rate of the loaded audio data is equal to the sample rate expected by the processor.


In [ ]:
# format suitable
#the code is using the Wav2Vec 2.0 processor (processor) to convert the raw audio waveform (waveform) into a format suitable for input to the Wav2Vec 2.0 model.

# used for debug: Print waveform shape for debugging
#print("Waveform shape:", waveform.shape)

# Ensure mono channel
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Ensure correct shape for model input
inputs = processor(waveform.squeeze(), sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt")
#used squeeze to give right matrix for inpput:torch.Size([1, 4865366])
# Print input shape for debugging
print("Input shape:", inputs.input_values.shape)

# Perform inference
with torch.no_grad():
    outputs = model(inputs.input_values)




Input shape: torch.Size([1, 4865366])


1. for this one exapmle task. it is one "No Training Updates:"
which means: During inference, the model's parameters are fixed and not updated.
2. in this application of the model , that gradient calculations and parameter updates, which are necessary during training, are not needed. to realize this , torch.no_grad()are used;
3.  torch.no_grad()  is a context manager provided by PyTorch. It temporarily sets all the requires_grad flags to False, which disables gradient calculation. This is useful during inference because we don't need gradients and it helps to save memory and improve performence.


In [ ]:
# Get the last hidden states
last_hidden_states = outputs.last_hidden_state
# Print shape of last_hidden_states
print("Last hidden states shape:", last_hidden_states.shape)

# Print the last_hidden_states
print("Last hidden states:", last_hidden_states)

NameError: name 'outputs' is not defined

last hidden states is final hidden state of a model after processing an input sequence.
in this task, the  last hidden states is our expected transcript form example m4a file

# use only for seveal m4a file(don't used for only one file)

the code below should only run if these are several m4a file in the address:
"/content/drive/MyDrive/UMD BIOE CAPSTONE PROJECT/sample/Raw data/"

the code also including the smapling ckeck part

In [ ]:
import os


# Load Wav2Vec 2.0 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# Directory containing audio files
directory = "/content/drive/MyDrive/UMD BIOE CAPSTONE PROJECT/sample/Raw data/"

# Loop over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".m4a"):
        audio_path = os.path.join(directory, filename)

        # Load audio file
        waveform, sample_rate = torchaudio.load(audio_path)

        # Preprocess the audio (resampling if necessary)
        if sample_rate != processor.feature_extractor.sampling_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, processor.feature_extractor.sampling_rate)
            waveform = resampler(waveform)

        # Process the audio using the processor
        inputs = processor(waveform, sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt")

        # Perform inference using the model
        with torch.no_grad():
            outputs = model(**inputs)

        # Get the last hidden states
        last_hidden_states = outputs.last_hidden_state
        print(f"Processed audio file: {audio_path}")

In [ ]:

from fairseq.models.wav2vec import Wav2VecModel

# Load the pretrained model
model = Wav2VecModel.from_pretrained('path_to_pretrained_model')

# Load your audio file
audio_input, _ = torchaudio.load('path_to_your_audio_file')

# Preprocess the audio (resampling, normalization, etc.)
# Ensure the audio matches the requirements of the pretrained model

# Transcribe the audio
transcription = model.transcribe(audio_input)

print("Transcription:", transcription)